In [1847]:
import csv
import os
import ast
import json
import pandas as pd
import numpy as np
import re
from git import Repo
import nltk
from datetime import datetime
from itertools import combinations
import hashlib
import math
from pydriller import Repository
import yaml
from collections import Counter
from scipy.stats import entropy
import ansiblemetrics
import puppetparser
from puppetparser.parser import parse
from ansiblelint import runner
from radon.metrics import Halstead
from radon.visitors import HalsteadVisitor
import statistics
import networkx as nx
import matplotlib.pyplot as plt

from ansiblemetrics.playbook.num_parameters import NumParameters
from ansiblemetrics.playbook.num_vars import NumVars
from ansiblemetrics.playbook.avg_play_size import AvgPlaySize
from ansiblemetrics.playbook.avg_task_size import AvgTaskSize
from ansiblemetrics.playbook.num_plays import NumPlays
from ansiblemetrics.playbook.num_tasks import NumTasks
from ansiblemetrics.playbook.num_unique_names import NumUniqueNames


# download required data
import ssl
import radon
import subprocess
   
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Initialize English stop words and lemmatizer
english_stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Specify the folder path containing the CSV files
#icinga2
projects1 = ['ansible-collection-hardening',
'iiab','Infra',
'ansible-hardening',
'postgresql_cluster',
'attack_range',
'trellis',
'ansible-nas',
'matrix-docker-ansible-deploy',
'algo','puppet-tp',
'puppet-icinga2',
'puppet-prometheus',
'eyp-systemd',
'puppet-hpc',
'control-repo',
'talend-cloud-installer',
'secure_windows',
'puppet-tripleo',
'puppet-openstack-integration']


projects = [
'iiab']
path = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_7/Projects/"
#matrix-docker-ansible-deploy, secure_windows, control-repo
median_indexes = ['Precision_based']
tops = [5]
algos= ['GA']
# features_files_coupling is the number of co-modified files divided by the total number of different files the iac file has changed with in the past
Output="/Users/narjes/Documents/ETS/Contrib5-Recommendation/ISA/FinalExperiment/_ISA_Features_L5_Ansible_1.csv"
with open(Output, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['instances', 'source', 'feature_iac_file_loc','feature_iac_file_churn','feature_iac_file_dependency','feature_nb_files','feature_nb_iac_files','feature_nb_noniac_files','feature_avg_files_per_commit','feature_avg_iac_files_per_commit','feature_avg_noniac_files_per_commit','feature_file_avg_history','feature_file_std_history','feature_nb_content_sim_files','feature_nb_path_sim_files','feature_avg_path_sim_files','feature_std_path_sim_files','feature_avg_content_sim_files','feature_std_content_sim_files','feature_Ratio_out_of_cofiles_developers','feature_Ratio_out_of_all_developers','feature_files_coupling','feature_files_nb_parameters','feature_files_nb_variables','feature_files_avg_play_size','feature_files_avg_task_size','feature_files_num_plays','feature_files_num_tasks','feature_files_num_commands','feature_files_num_blocks','feature_files_num_Distinctmodules','feature_files_num_ExternalModules','feature_files_num_FactModules','feature_files_num_Paths','feature_files_num_Roles','feature_files_num_DeprecatedModules','feature_num_ImportedRoles','feature_num_ImportedPlaybooks','feature_num_ImportedTasks','feature_num_IncludedRoles','feature_num_IncludedVars','feature_num_Includes','feature_num_IncludedTasks','feature_files_num_normalized_resources','feature_Graph_History_std_degree_centrality','feature_Graph_History_network_density','feature_Graph_History_Degree_centrality','feature_Graph_History_betweenness_centrality','feature_Graph_History_closeness_centrality','feature_Graph_ContentSimilarity_std_degree_centrality','feature_Graph_ContentSimilarity_network_density','feature_Graph_ContentSimilarity_Degree_centrality','feature_Graph_ContentSimilarity_betweenness_centrality','feature_Graph_ContentSimilarity_closeness_centrality','feature_Graph_PathSimilarity_std_degree_centrality','feature_Graph_PathSimilarity_network_density','feature_Graph_PathSimilarity_Degree_centrality','feature_Graph_PathSimilarity_betweenness_centrality','feature_Graph_PathSimilarity_closeness_centrality','algo_GA','algo_PSO','algo_RS','algo_SA','algo_HIST','algo_SIM'])
      

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/narjes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/narjes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/narjes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [1848]:
#writer.writerow(['instances', 'source','feature_type', 'feature_nb_files', 'feature_nb_others','feature_nb_iac','feature_has_history','feature_avg_hist','feature_avg_sim','feature_std_hist','feature_std_sim','feature_nb_files_with_history' 'algo_GA','algo_PSO','algo_SA','algo_RS'])
      
def save_data(GA_precision, PSO_precision,RS_precision,SA_precision,HIST_precision,SIM_precision,commits,instances,feature_file_loc,feature_iac_file_churn,feature_file_dependency,feature_nb_files,feature_nb_iac_files,feature_nb_noniac_files,feature_avg_files_per_commit,feature_avg_iac_files_per_commit,feature_avg_noniac_files_per_commit,feature_file_avg_history,feature_file_std_history,feature_nb_content_sim_files,feature_nb_path_sim_files,feature_avg_path_sim_files,feature_std_path_sim_files,feature_avg_content_sim_files,feature_std_content_sim_files,feature_Ratio_out_of_cofiles_developers,feature_Ratio_out_of_all_developers,feature_files_coupling,feature_files_nb_parameters,feature_files_nb_variables,feature_files_avg_play_size,feature_files_avg_task_size,feature_files_num_plays,feature_files_num_tasks,feature_files_num_commands,feature_files_num_blocks,feature_files_num_Distinctmodules,feature_files_num_ExternalModules,feature_files_num_FactModules,feature_files_num_Paths,feature_files_num_Roles,feature_files_num_DeprecatedModules,feature_num_NumImportedRoles,feature_num_ImportedPlaybooks,feature_num_NumImportedTasks,feature_num_NumIncludedRoles,feature_num_NumIncludedVars,feature_num_NumIncludes,feature_num_NumIncludedTasks,feature_files_num_normalized_resources,feature_Graph_std_degree_centrality,feature_Graph_network_density,feature_Graph_Avg_Degree_centrality,feature_Graph_Avg_betweenness_centrality,feature_Graph_Avg_closeness_centrality,feature_Graph_ContentSimilarity_std_degree_centrality,feature_Graph_ContentSimilarity_network_density,feature_Graph_ContentSimilarity_Avg_Degree_centrality,feature_Graph_ContentSimilarity_Avg_betweenness_centrality,feature_Graph_ContentSimilarity_Avg_closeness_centrality,feature_Graph_PathSimilarity_std_degree_centrality,feature_Graph_PathSimilarity_network_density,feature_Graph_PathSimilarity_Avg_Degree_centrality,feature_Graph_PathSimilarity_Avg_betweenness_centrality,feature_Graph_PathSimilarity_Avg_closeness_centrality,seed):
    counter = seed
    for i in range(len(instances)):
        #print(counter)
        instances[i] = counter
        counter += 1
    with open(Output, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        for i in range(len(commits)):
            writer.writerow([instances[i], commits[i], feature_file_loc[i],feature_iac_file_churn[i],feature_file_dependency[i],feature_nb_files[i],feature_nb_iac_files[i],feature_nb_noniac_files[i],feature_avg_files_per_commit[i],feature_avg_iac_files_per_commit[i],feature_avg_noniac_files_per_commit[i],feature_file_avg_history[i],feature_file_std_history[i],feature_nb_content_sim_files[i],feature_nb_path_sim_files[i],feature_avg_path_sim_files[i],feature_std_path_sim_files[i],feature_avg_content_sim_files[i],feature_std_content_sim_files[i],feature_Ratio_out_of_cofiles_developers[i],feature_Ratio_out_of_all_developers[i],feature_files_coupling[i],feature_files_nb_parameters[i],feature_files_nb_variables[i],feature_files_avg_play_size[i],feature_files_avg_task_size[i],feature_files_num_plays[i],feature_files_num_tasks[i],feature_files_num_commands[i],feature_files_num_blocks[i],feature_files_num_Distinctmodules[i],feature_files_num_ExternalModules[i],feature_files_num_FactModules[i],feature_files_num_Paths[i],feature_files_num_Roles[i],feature_files_num_DeprecatedModules[i],feature_num_NumImportedRoles[i],feature_num_ImportedPlaybooks[i],feature_num_NumImportedTasks[i],feature_num_NumIncludedRoles[i],feature_num_NumIncludedVars[i],feature_num_NumIncludes[i],feature_num_NumIncludedTasks[i],feature_files_num_normalized_resources[i],feature_Graph_std_degree_centrality[i],feature_Graph_network_density[i],feature_Graph_Avg_Degree_centrality[i],feature_Graph_Avg_betweenness_centrality[i],feature_Graph_Avg_closeness_centrality[i],feature_Graph_ContentSimilarity_std_degree_centrality[i],feature_Graph_ContentSimilarity_network_density[i],feature_Graph_ContentSimilarity_Avg_Degree_centrality[i],feature_Graph_ContentSimilarity_Avg_betweenness_centrality[i],feature_Graph_ContentSimilarity_Avg_closeness_centrality[i],feature_Graph_PathSimilarity_std_degree_centrality[i],feature_Graph_PathSimilarity_network_density[i],feature_Graph_PathSimilarity_Avg_Degree_centrality[i],feature_Graph_PathSimilarity_Avg_betweenness_centrality[i],feature_Graph_PathSimilarity_Avg_closeness_centrality[i],GA_precision[i],PSO_precision[i],RS_precision[i],SA_precision[i],HIST_precision[i],SIM_precision[i]])

    #print("Values written to", Output)
    return counter

In [1849]:
def get_old_or_new_name(iacfile, commit_hash, repo):
    churn=0
    for commit in Repository(repo, single= commit_hash).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == iacfile:
                churn = modified_file.added_lines+modified_file.deleted_lines
                return churn
    return None  # Return None if the file is not found in any commit

In [1850]:
def file_churn(iacfile, commit, repo):
    _total_iac = 0
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)

    # Check if the old name is in the commit's file list
    if _text in commit.stats.files:
        _total_iac += commit.stats.files[_text]['insertions'] + commit.stats.files[_text]['deletions']
    else:
        # Get the old or new name using PyDriller
        churn = get_old_or_new_name(_text, commit, repo)
   
        _total_iac += churn
       
    return _total_iac

In [1851]:
def get_old_or_new_name_loc(iacfile, commit_hash, repo):
    size=0
    for commit in Repository(repo, single= commit_hash).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == iacfile:
                size = modified_file.nloc
                return size
    return None  # Return None if the file is not found in any commit

In [1852]:
def file_loc(iacfile, commit, repo):
    file_loc = 0
    num_lines=0
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)

    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == _text:
                num_lines = modified_file.content.decode('utf-8').count('\n') 
            elif modified_file.old_path == _text:
                num_lines = modified_file.content.decode('utf-8').count('\n') 
    file_loc =num_lines
    return file_loc



In [1853]:
#size = modified_file.complexity
def file_puppet_complexity(iacfile, commit, repo):
    file_complexity = 0
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)
    puppet_reserved_words = ['class', 'define', 'node', 'package', 'file','service']
    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == _text:

                content_str = modified_file.content.decode('utf-8')
            
                loc = len([line for line in content_str.splitlines() if line.strip()])
                #print(loc)
                # Tokenize the contents of the file
                words = re.findall(r'\b\w+\b', content_str)
                
            
                #loc = len([line for line in modified_file.content.splitlines() if line.strip()])
                #print(loc)
                # Tokenize the contents of the file
                #words = re.findall(r'\b\w+\b', modified_file.content)

                # Filter out reserved words
                filtered_words = set(word for word in words if word in puppet_reserved_words)
                #print(f'Resources: ',filtered_words)

                normalized=      len(filtered_words)/loc  
                not_normalized= len(filtered_words)

                
            elif modified_file.old_path == _text:
                loc = len([line for line in modified_file.content.splitlines() if line.strip()])
                #print(loc)
                # Tokenize the contents of the file
                words = re.findall(r'\b\w+\b', modified_file.content)

                # Filter out reserved words
                filtered_words = set(word for word in words if word in puppet_reserved_words)
                #print(f'Resources: ',filtered_words)

                normalized=      len(filtered_words)/loc  
                not_normalized= len(filtered_words)
  
    file_complexity =normalized
    return file_complexity

In [1854]:
import radon
import radon.complexity
import radon.metrics
from radon.complexity import average_complexity

def calculate_complexity(iacfile, commit, repo):
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)
    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == _text:

                content_str = modified_file.content.decode('utf-8')
                content_str = """{}""".format(content_str)
                #print(content_str)
                try:
                    # Parse the code into an AST
                    tree = ast.parse(content_str, type_comments=True)

                    # Extract the blocks (functions, classes) from the AST
                    blocks = [node for node in tree.body if hasattr(node, 'lineno')]

                    # Calculate the cyclomatic complexity
                    complexity = average_complexity(blocks)

                    return complexity
                except Exception as e:
                    # Handle any errors that occur during the complexity calculation
                    print(f"Error calculating complexity: {str(e)}")
                    return None

            elif modified_file.old_path == _text: 
                content_str = modified_file.content.decode('utf-8')
                content_str = """{}""".format(content_str) 
                try:
                    # Parse the code into an AST
                    tree = ast.parse(content_str, type_comments=True)

                    # Extract the blocks (functions, classes) from the AST
                    blocks = [node for node in tree.body if hasattr(node, 'lineno')]

                    # Calculate the cyclomatic complexity
                    complexity = average_complexity(blocks)

                    return complexity
                except Exception as e:
                    # Handle any errors that occur during the complexity calculation
                    print(f"Error calculating complexity: {str(e)}")
                    return None
        


In [1855]:
def calculate_complexity1(playbook_file):
    runner = Runner(playbook_file)
    return runner

In [1856]:
def file_ansible_complexity11(iacfile, commit, repo):
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)
    complexity_metrics = calculate_complexity(_text)
    #print(complexity_metrics)

    return complexity_metrics

In [1857]:
#size = modified_file.complexity
def file_ansible_complexity_OLD(iacfile, commit, repo):
    file_complexity = 0
    plays = 0
    tasks = 0
    handlers = 0
    roles = 0
    variables = 0
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)
    
    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == _text:
                
              loc = len([line for line in modified_file.content.splitlines() if line.strip()])
              
              try:  
                data = yaml.safe_load(modified_file.content)
                #print(data)
                if isinstance(data, list):
                    plays = len(data)
                    for play in data:
                        #print(play)
                        tasks += len(play.get('tasks', []))
                        handlers += len(play.get('handlers', []))
                        roles += len(play.get('roles', []))
                        variables += len(play.get('vars', []))
              except yaml.YAMLError as exc:
                print(f"Error parsing YAML file {modified_file.new_path}: {exc}")
     
              normalized_complexity= (plays+tasks+handlers+roles+variables)/loc
              all_compexlity= plays+tasks+handlers+roles+variables
 
                
            elif modified_file.old_path == _text:
                loc = len([line for line in modified_file.content.splitlines() if line.strip()])
                data = yaml.safe_load(modified_file.content)
                #print(data)
                if isinstance(data, list):
                    plays = len(data)
                    for play in data:
                        #print(play)
                        tasks += len(play.get('tasks', []))
                        handlers += len(play.get('handlers', []))
                        roles += len(play.get('roles', []))
                        variables += len(play.get('vars', []))
                    
                normalized_complexity= (plays+tasks+handlers+roles+variables)/loc
                all_compexlity= plays+tasks+handlers+roles+variables

    file_complexity =normalized_complexity
    return file_complexity

In [1858]:
from ansiblemetrics.playbook.num_imported_playbooks import NumImportedPlaybooks
from ansiblemetrics.playbook.num_imported_roles import NumImportedRoles
from ansiblemetrics.playbook.num_imported_tasks import NumImportedTasks
from ansiblemetrics.playbook.num_included_roles import NumIncludedRoles
from ansiblemetrics.playbook.num_included_vars import NumIncludedVars
from ansiblemetrics.playbook.num_includes import NumIncludes
from ansiblemetrics.playbook.num_included_tasks import NumIncludedTasks

from ansiblemetrics.playbook.avg_play_size import AvgPlaySize
from ansiblemetrics.playbook.avg_task_size import AvgTaskSize
from ansiblemetrics.playbook.num_plays import NumPlays
from ansiblemetrics.playbook.num_tasks import NumTasks
from ansiblemetrics.playbook.num_commands import NumCommands
from ansiblemetrics.playbook.num_blocks import NumBlocks
from ansiblemetrics.playbook.num_distinct_modules import NumDistinctModules
from ansiblemetrics.playbook.num_external_modules import NumExternalModules
from ansiblemetrics.playbook.num_fact_modules import NumFactModules
from ansiblemetrics.playbook.num_paths import NumPaths
from ansiblemetrics.playbook.num_roles import NumRoles
from ansiblemetrics.playbook.num_vars import NumVars
from ansiblemetrics.playbook.num_deprecated_modules import NumDeprecatedModules
#from ansiblemetrics.lines_metric import LinesMetric

In [1859]:
def file_ansible_resources(iacfile, commit, repo):
    file_dependencies = 0
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)
    
    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == _text:

                if modified_file.content is not None:
                    num_lines = modified_file.content.decode('utf-8').count('\n') 
                    #loc = len([line for line in modified_file.content.splitlines() if line.strip()])
                    num_NumPlays = NumPlays(modified_file.content).count()
                    num_NumTasks = NumTasks(modified_file.content).count()
                    num_NumRoles = NumRoles(modified_file.content).count()
                    num_NumBlocks = NumBlocks(modified_file.content).count()
                    num_NumFactModules = NumFactModules(modified_file.content).count()
                    #num_LinesMetric = LinesMetric(modified_file.content).count()
                    resources = (num_NumPlays+num_NumTasks+num_NumRoles+num_NumBlocks+num_NumFactModules)/num_lines

                
            elif modified_file.old_path == _text:
                if modified_file.content is not None:
                    num_lines = modified_file.content.decode('utf-8').count('\n') 
                    #loc = len([line for line in modified_file.content.splitlines() if line.strip()])
                    num_NumPlays = NumPlays(modified_file.content).count()
                    num_NumTasks = NumTasks(modified_file.content).count()
                    num_NumRoles = NumRoles(modified_file.content).count()
                    num_NumBlocks = NumBlocks(modified_file.content).count()
                    num_NumFactModules = NumFactModules(modified_file.content).count()
                    #num_LinesMetric = LinesMetric(modified_file.content).count()
                    resources = (num_NumPlays+num_NumTasks+num_NumRoles+num_NumBlocks+num_NumFactModules)/num_lines

    #file_depen =file_dependencies
    return resources

In [1860]:
def iac_dependency(iacfile, commit, repo):
    file_dependencies = 0
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    _text = re.sub(r'^.*?/', '', _string)
    
    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == _text:
                if modified_file.content is not None:
                    num_ImportedPlaybooks = NumImportedPlaybooks(modified_file.content).count()
                    num_NumImportedRoles = NumImportedRoles(modified_file.content).count()
                    num_NumImportedTasks = NumImportedTasks(modified_file.content).count()
                    num_NumIncludedRoles = NumIncludedRoles(modified_file.content).count()
                    num_NumIncludedVars = NumIncludedVars(modified_file.content).count()
                    num_NumIncludes = NumIncludes(modified_file.content).count()
                    num_NumIncludedTasks = NumIncludedTasks(modified_file.content).count()
            
                    #import_count = len(re.findall(r'.*import.*', modified_file.content.decode('utf-8')))
                    #include_count = len(re.findall(r'.*include.*', modified_file.content.decode('utf-8')))
                    #require_count = len(re.findall(r'.*require.*', modified_file.content.decode('utf-8')))

                    file_dependencies = num_ImportedPlaybooks+num_NumImportedRoles+num_NumImportedTasks+num_NumIncludedRoles+num_NumIncludedVars+num_NumIncludes+num_NumIncludedTasks

                
            elif modified_file.old_path == _text:
                if modified_file.content is not None:
                    num_ImportedPlaybooks = NumImportedPlaybooks(modified_file.content).count()
                    num_NumImportedRoles = NumImportedRoles(modified_file.content).count()
                    num_NumImportedTasks = NumImportedTasks(modified_file.content).count()
                    num_NumIncludedRoles = NumIncludedRoles(modified_file.content).count()
                    num_NumIncludedVars = NumIncludedVars(modified_file.content).count()
                    num_NumIncludes = NumIncludes(modified_file.content).count()
                    num_NumIncludedTasks = NumIncludedTasks(modified_file.content).count()

                    file_dependencies = num_ImportedPlaybooks+num_NumImportedRoles+num_NumImportedTasks+num_NumIncludedRoles+num_NumIncludedVars+num_NumIncludes+num_NumIncludedTasks


    #file_depen =file_dependencies
    return file_dependencies,num_ImportedPlaybooks,num_NumImportedRoles,num_NumImportedTasks,num_NumIncludedRoles,num_NumIncludedVars,num_NumIncludes,num_NumIncludedTasks



In [1861]:
def count_parameters(path):
        
        parsed_scripts, comments = parse(path)
        params = []
        #print(parsed_scripts)
        for parsed_script in parsed_scripts:
          if hasattr(parsed_script, 'resources'):
            for resource in parsed_script.resources:
                for param in resource.parameters:
                    params.append(param.name)
        return len(params)

In [1862]:
def puppet_num_variables(iacfile, commit, repo):
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    #print(_string)
    manifest_path = re.sub(r'^.*?/', '', _string)

    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == manifest_path:
                if modified_file.content is not None:
                    vars_count = len(re.findall(r'\$(\w+)\s*=(?!.*(?:,|\)))', modified_file.content.decode('utf-8')))


            elif modified_file.old_path == manifest_path:
                if modified_file.content is not None:
                    vars_count = len(re.findall(r'\$(\w+)\s*=(?!.*(?:,|\)))', modified_file.content.decode('utf-8')))


    #print(f"Number of variables: {vars_count}")
    return vars_count

In [1863]:
def puppet_num_parameters(iacfile, commit, repo):
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    #print(_string)
    manifest_path = re.sub(r'^.*?/', '', _string)

    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == manifest_path:
                if modified_file.content is not None:
                    params_count = len(re.findall(r'\$(\w+)\s*=\s*(.*)\s*(?=,|\))', modified_file.content.decode('utf-8')))


            elif modified_file.old_path == manifest_path:
                if modified_file.content is not None:
                    params_count = len(re.findall(r'\$(\w+)\s*=\s*(.*)\s*(?=,|\))', modified_file.content.decode('utf-8')))


    #print(f"Number of parameters: {params_count}")
    return params_count

In [1864]:
def ansible_quality(iacfile, commit, repo):
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    playbook_path = re.sub(r'^.*?/', '', _string)

    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == playbook_path:
                if modified_file.content is not None:
                    avg_play_size = AvgPlaySize(modified_file.content).count()
                    avg_task_size = AvgTaskSize(modified_file.content).count()
                    num_plays = NumPlays(modified_file.content).count()
                    num_tasks = NumTasks(modified_file.content).count()
                    num_commands = NumCommands(modified_file.content).count()
                    num_blacks = NumBlocks(modified_file.content).count()
                    num_Distinctmodules = NumDistinctModules(modified_file.content).count()
                    num_ExternalModules = NumExternalModules(modified_file.content).count()
                    num_FactModules = NumFactModules(modified_file.content).count()
                    num_Paths = NumPaths(modified_file.content).count()
                    num_Roles = NumRoles(modified_file.content).count()
                    num_DeprecatedModules = NumDeprecatedModules(modified_file.content).count()
                    #num_unique_names = NumUniqueNames(modified_file.content).count()

            elif modified_file.old_path == playbook_path:
                if modified_file.content is not None:
                    avg_play_size = AvgPlaySize(modified_file.content).count()
                    avg_task_size = AvgTaskSize(modified_file.content).count()
                    num_plays = NumPlays(modified_file.content).count()
                    num_tasks = NumTasks(modified_file.content).count()
                    num_commands = NumCommands(modified_file.content).count()
                    num_blacks = NumBlocks(modified_file.content).count()
                    num_Distinctmodules = NumDistinctModules(modified_file.content).count()
                    num_ExternalModules = NumExternalModules(modified_file.content).count()
                    num_FactModules = NumFactModules(modified_file.content).count()
                    num_Paths = NumPaths(modified_file.content).count()
                    num_Roles = NumRoles(modified_file.content).count()
                    num_DeprecatedModules = NumDeprecatedModules(modified_file.content).count()
                    #num_unique_names = NumUniqueNames(modified_file.content).count()

    return avg_play_size,avg_task_size,num_plays,num_tasks,num_commands,num_blacks,num_Distinctmodules,num_ExternalModules,num_FactModules,num_Paths,num_Roles,num_DeprecatedModules

In [1865]:
def ansible_num_variables(iacfile, commit, repo):
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    playbook_path = re.sub(r'^.*?/', '', _string)

    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == playbook_path:
                if modified_file.content is not None:
                    vars_count = NumVars(modified_file.content).count()

            elif modified_file.old_path == playbook_path:
                if modified_file.content is not None:
                    vars_count = NumVars(modified_file.content).count()

    return vars_count

In [1866]:
def ansible_num_parameters(iacfile, commit, repo):
    _string = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', iacfile)
    playbook_path = re.sub(r'^.*?/', '', _string)

    for commit in Repository(repo, single= commit).traverse_commits():
        for modified_file in commit.modified_files:
            if modified_file.new_path == playbook_path:
                if modified_file.content is not None:
                    params_count = NumParameters(modified_file.content).count()

            elif modified_file.old_path == playbook_path:
                if modified_file.content is not None:
                    params_count = NumParameters(modified_file.content).count()

    
    #params_count = NumParameters(modified_file.content)

    #print(f"Number of parameters: {params_count}")
    return params_count

In [1867]:
def extract_files(csv_path, commit_hash, file_x):
    df = pd.read_csv(csv_path, header=None)

    commit_str = str(commit_hash)
    commit_index = df.iloc[:, 0].tolist().index(commit_str)

    files_set = set()
    for index in range(commit_index ):
        row = df.iloc[index]
        files = row[1:].dropna().values  # exclude NaN values
        if file_x in files:
            files_set.update(set(files) - {file_x})

    return list(files_set)

In [1868]:
def number_files(filename,commit_obj,csv_hist_path, path,test_commits):
    nb_files=0

    df = pd.read_csv(csv_hist_path, index_col=0)
    ## process filenae
    matching_files_row = [index for index in df.index if filename in index]
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]
    # Count the number of files with a value > 0
    nb_files = (column_values > 0).sum()
    #print(nb_files)

    list_files_from_test_set_until_commitX=extract_files(test_commits, commit_obj, filename)
    #print(list_files_from_test_set_until_commitX)

    missing_files = [file for file in list_files_from_test_set_until_commitX if file not in column_values.index]
    #print(missing_files)

    nb_files=nb_files+len(missing_files)

    
    return nb_files

In [1869]:
def number_iac_files(filename,commit_obj,csv_hist_path, path,test_commits):
    nb_files=0

    df = pd.read_csv(csv_hist_path, index_col=0)
    ## process filenae
    matching_files_row = [index for index in df.index if filename in index]
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]

    
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]
    #print(column_values)

    # Remove files with '(Other)' in their names
    #column_values = column_values[~column_values.index.str.contains('(Other)')]
    column_values = column_values[~column_values.index.str.contains('(Other)', regex=False)]

    #print(column_values)
    # Count the number of files with a value > 0
    nb_files = (column_values > 0).sum()
    #print(nb_files)

    list_files_from_test_set_until_commitX=extract_files(test_commits, commit_obj, filename)
    list_files_from_test_set_until_commitX = [file for file in list_files_from_test_set_until_commitX if '(Other)' not in file]

    #print(list_files_from_test_set_until_commitX)

    missing_files = [file for file in list_files_from_test_set_until_commitX if file not in column_values.index]
    #print(missing_files)

    nb_files=nb_files+len(missing_files)

    
    return nb_files

In [1870]:
def number_noniac_files(filename,commit_obj,csv_hist_path, path,test_commits):
    nb_files=0

    df = pd.read_csv(csv_hist_path, index_col=0)
    ## process filenae
    matching_files_row = [index for index in df.index if filename in index]
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]

    
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]
    #print(column_values)

    # Remove files with '(Other)' in their names
    #column_values = column_values[~column_values.index.str.contains('(IaC)')]
    column_values = column_values[~column_values.index.str.contains('(IaC)', regex=False)]

    #print(column_values)
    # Count the number of files with a value > 0
    nb_files = (column_values > 0).sum()
    #print(nb_files)

    list_files_from_test_set_until_commitX=extract_files(test_commits, commit_obj, filename)
    list_files_from_test_set_until_commitX = [file for file in list_files_from_test_set_until_commitX if '(IaC)' not in file]

    #print(list_files_from_test_set_until_commitX)

    missing_files = [file for file in list_files_from_test_set_until_commitX if file not in column_values.index]
    #print(missing_files)

    nb_files=nb_files+len(missing_files)

    
    return nb_files



In [1871]:
def extract_files_from_train(csv_path, file_x):
    df = pd.read_csv(csv_path, header=None)

    
    # Find all indices where the file exists
    file_indices = [i for i, row in enumerate(df.values) for x in row if x == file_x]
    #print(file_indices)
    # Count the number of columns for each index
    column_counts = []
    for i, row in enumerate(df.values):
        if file_x in row:
            count = 0
            for val in row:
                if str(val) != 'nan' and str(val) != '':
                    count += 1
            
                column_counts.append(count - 1)
    #print(column_counts)
    

    return column_counts

In [1872]:
def extract_files_avg(csv_path, commit_hash, file_x):
    df = pd.read_csv(csv_path, header=None)

    commit_str = str(commit_hash)
    #print(commit_str)
    #commit_index = [index for index in df.index if commit_str in index]
   # commit_index = df.index.get_loc(commit_str)
    #commit_index = df.loc[df.iloc[:, 0] == commit_str].index[0]
    commit_index = df.iloc[:, 0].tolist().index(commit_str)

    column_counts = []
    #print(commit_index)
    #max_index = max(commit_index)
    for index in range(commit_index ):
        row = df.iloc[index]
        files = row[1:].dropna().values  # exclude NaN values
        if file_x in files:
            count = 0
            for val in row:
                if str(val) != 'nan' and str(val) != '':
                    count += 1
            column_counts.append(count - 1)
        

    return column_counts

In [1873]:
def Avg_files(filename,commit_obj,csv_hist_path, path,train_commits,test_commits):
    avg_files=0

    avg_from_train=extract_files_from_train(train_commits,filename)
    #print(avg_from_train)

    list_files_from_test_set_until_commitX=extract_files_avg(test_commits, commit_obj, filename)
    #print(list_files_from_test_set_until_commitX)


    merged_list = avg_from_train + list_files_from_test_set_until_commitX
    
    avg_files=np.mean(merged_list)
    #print(avg_files)
    return avg_files


In [1874]:
def extract_iac_files_from_train(csv_path, file_x):
    df = pd.read_csv(csv_path, header=None)

    
    # Find all indices where the file exists
    file_indices = [i for i, row in enumerate(df.values) for x in row if x == file_x]
    #print(file_indices)
    # Count the number of columns for each index
    column_counts = []
    for i, row in enumerate(df.values):
        if file_x in row:
            count=0
            other_count = 0
            for val in row:
                if str(val) != 'nan' and str(val) != '':
                    count += 1
                    if "(Other)" in str(val):
                        other_count += 1
            column_counts.append(count - 1 - other_count)
    #print(column_counts)
    

    return column_counts

In [1875]:
def extract_iac_files_avg(csv_path, commit_hash, file_x):
    df = pd.read_csv(csv_path, header=None)

    commit_str = str(commit_hash)
    commit_index = df.iloc[:, 0].tolist().index(commit_str)

    column_counts = []
    for index in range(commit_index ):
        row = df.iloc[index]
        files = row[1:].dropna().values  # exclude NaN values
        if file_x in files:
            count = 0
            other_count = 0
            for val in row:
                if str(val) != 'nan' and str(val) != '':
                    count += 1
                    if "(Other)" in str(val):
                        other_count += 1
            column_counts.append(count - 1 - other_count)
        

    return column_counts

In [1876]:
def Avg_iac_files(filename,commit_obj,csv_hist_path, path,train_commits,test_commits):
    avg_files=0

    avg_from_train=extract_iac_files_from_train(train_commits,filename)
    #print(avg_from_train)

    list_files_from_test_set_until_commitX=extract_iac_files_avg(test_commits, commit_obj, filename)
    #print(list_files_from_test_set_until_commitX)


    merged_list = avg_from_train + list_files_from_test_set_until_commitX
    
    avg_files=np.mean(merged_list)
    #print(avg_files)
    return avg_files

In [1877]:
def extract_noniac_files_from_train(csv_path, file_x):
    df = pd.read_csv(csv_path, header=None)

    
    # Find all indices where the file exists
    file_indices = [i for i, row in enumerate(df.values) for x in row if x == file_x]
    #print(file_indices)
    # Count the number of columns for each index
    column_counts = []
    for i, row in enumerate(df.values):
        if file_x in row:
            count=0
            other_count = 0
            for val in row:
                if str(val) != 'nan' and str(val) != '':
                    count += 1
                    if "(IaC)" in str(val):
                        other_count += 1
            column_counts.append(count - 1 - other_count)
    #print(column_counts)
    

    return column_counts

In [1878]:
def extract_noniac_files_avg(csv_path, commit_hash, file_x):
    df = pd.read_csv(csv_path, header=None)

    commit_str = str(commit_hash)
    commit_index = df.iloc[:, 0].tolist().index(commit_str)

    column_counts = []
    for index in range(commit_index ):
        row = df.iloc[index]
        files = row[1:].dropna().values  # exclude NaN values
        if file_x in files:
            count = 0
            other_count = 0
            for val in row:
                if str(val) != 'nan' and str(val) != '':
                    count += 1
                    if "(IaC)" in str(val):
                        other_count += 1
            column_counts.append(count - 1 - other_count)
        

    return column_counts

In [1879]:
def Avg_noniac_files(filename,commit_obj,csv_hist_path, path,train_commits,test_commits):
    avg_files=0

    avg_from_train=extract_noniac_files_from_train(train_commits,filename)
    #print(avg_from_train)

    list_files_from_test_set_until_commitX=extract_noniac_files_avg(test_commits, commit_obj, filename)
    #print(list_files_from_test_set_until_commitX)


    merged_list = avg_from_train + list_files_from_test_set_until_commitX
    
    avg_files=np.mean(merged_list)
    #print(avg_files)
    return avg_files

In [1880]:
def iac_avg_dependency(iacfile, commitOrig, repo,path,train_commits,test_commits):
    dependencies_per_commit=[]
    #extract past commits were file has changed and get its content in each commit
    df = pd.read_csv(train_commits, header=None)
    # Find all indices where the file exists
    file_indices = [i for i, row in enumerate(df.values) for x in row if x == iacfile]
    commits = [df.iloc[i, 0] for i in file_indices]

    for commit in commits:
        #print(commit)
        commit_obj = repo.commit(commit)

 
        file_dependencies=iac_dependency(iacfile,commit_obj,path)
        dependencies_per_commit.append(file_dependencies)
        

    #extract test commit  were file has changed before current commit and get its content
    df = pd.read_csv(test_commits, header=None)

    commit_str = str(commitOrig)
    commit_index = df.iloc[:, 0].tolist().index(commit_str)

    for index in range(commit_index ):
        row = df.iloc[index]
        files = row[1:].dropna().values  # exclude NaN values
        if iacfile in files:
            comm=row[0]
            #print(comm)
            commit_obj = repo.commit(comm)
            file_dependencies=iac_dependency(iacfile,commit_obj,path)
            dependencies_per_commit.append(file_dependencies)
            
            
        
    file_depen =np.mean(dependencies_per_commit)
    return file_depen

In [1881]:
def extract_files_avg_hist(csv_path, commit_hash, file_x):
    df = pd.read_csv(csv_path, header=None)

    commit_str = str(commit_hash)
    commit_index = df.iloc[:, 0].tolist().index(commit_str)

    files_set = []
    for index in range(commit_index ):
        row = df.iloc[index]
        files = row[1:].dropna().values  # exclude NaN values
        if file_x in files:
            files_set.extend([file for file in files if file != file_x])

    return files_set

In [1882]:
def file_avg_history(filename,commit_obj,csv_hist_path, path,test_commits):
    avg_hist=0
    std_hist=0
    df = pd.read_csv(csv_hist_path, index_col=0)
    ## process filenae
    matching_files_row = [index for index in df.index if filename in index]
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]

    
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]

    history_values = column_values.loc[column_values != 0].tolist()

    #print(history_values)

    list_files_from_test_set_until_commitX=extract_files_avg_hist(test_commits, commit_obj, filename)
    #if list_files_from_test_set_until_commitX:
        

    counts_list = [list_files_from_test_set_until_commitX.count(file) for file in set(list_files_from_test_set_until_commitX)]

    #print(counts_list)
        
        #print(list_files_from_test_set_until_commitX)

    hist_averages=history_values+counts_list
    #print(hist_averages)
    
    avg_hist=np.mean(hist_averages)
    std_hist=np.std(hist_averages)
    #print(avg_hist)
    #print(std_hist)
    return avg_hist, std_hist

In [1883]:
def process_file(filename):
    with open(filename, 'r', encoding="utf-8", errors="ignore") as file:
        content = file.read()

        # Tokenize the content
        words = word_tokenize(content.lower())

        # Filter out English stop words
        filtered_words = [word for word in words if word not in english_stopwords]

        # Lemmatize the filtered words
        lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

        # Join the lemmatized words back into text
        return ' '.join(lemmatized_words)

In [1884]:
def calculate_content_score(path1,path2):
    files = [process_file(path1), process_file(path2)]
    

    # Initialize TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform data into vectors
    tfidf_matrix = tfidf_vectorizer.fit_transform(files)

    # Calculate cosine similarity scores between the vectors of the two files
    scores = cosine_similarity(tfidf_matrix)
    #print(scores)
    # Output the cosine similarity score between file1 and file2
    cosine_similarity_score = scores[0, 1]
    return cosine_similarity_score

In [1885]:
def nb_content_sim_files(filename,commit_obj,csv_content_sim, path,train_commits,test_commits):
    nb_files=0

    df = pd.read_csv(csv_content_sim, index_col=0)
    modified_string = filename.replace("_(IaC)_", "", 1)
    ## process filenae
    #print(modified_string)
    matching_files_row = [index for index in df.index if modified_string in index]
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]

    
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]

    #print(column_values)
    nb_files = (column_values > 0.5).sum()
    #print(nb_files)


    list_files_from_test_set_until_commitX=extract_files(test_commits, commit_obj, filename)
    #print(list_files_from_test_set_until_commitX)

    missing_files = [file for file in list_files_from_test_set_until_commitX if file not in column_values.index]
    #print(missing_files)


    base_path1 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/"
    base_path2 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"   
    score=0
    scores = []
    for file in missing_files:
      ##print(file)
        modified_file = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', file)
       # print(modified_file)
        full_path1 = os.path.join(base_path1, modified_file)
        full_path2 = os.path.join(base_path2, modified_file)
        #print(full_path1)
        #print(full_path2)
        
        path1_exists = os.path.exists(full_path1)
        path2_exists = os.path.exists(full_path2)
        #print(path1_exists,path2_exists)

        # Determine which path to use based on the file's existence
        if path1_exists:
            #print("path1")
            path_to_use = base_path1
            
        elif path2_exists:
            #print("path2")
            path_to_use = base_path2
        
        else:
            scores.append(0.0)
            continue
        path2 = path_to_use + modified_file
        path1 = path_to_use + modified_string
        #print(path2)
        #print(path1)
        # Attempt to calculate the content score, handling FileNotFoundError
        try:
            score = calculate_content_score(path1, path2)
            #print(score)
            scores.append(score)
        except FileNotFoundError:
            # Handle the case where the file does not exist
            #print(f"One of the files was not found: {path1} or {path2}")
            # Decide on appropriate handling, such as appending a default score or logging
            scores.append(0.0)
    


    count = len([x for x in scores if x > 0.5])
    nb_files=nb_files+count
    #print(nb_files)
    
    return nb_files

In [1886]:
def calcualte_path_score(path1,path2):
    #print(path1,path2)
    file1_words = re.split('/|_|-', path1.lower()) 
    file2_words = re.split('/|_|-', path2.lower())
    max_num_words = max(len(file1_words), len(file2_words))  

    common_words = set(file1_words) & set(file2_words)

    total_unique_words = len(set(file1_words + file2_words)) 

    common_words_path = len(common_words) / total_unique_words
    return common_words_path

In [1887]:
def nb_path_sim_files(filename,commit_obj,csv_path_sim, path,train_commits,test_commits):
    nb_files=0

    df = pd.read_csv(csv_path_sim, index_col=0)
    #modified_string = filename.replace("_(IaC)_", "", 1)
    ## process filenae
    #print(modified_string)
    matching_files_row = [index for index in df.index if filename in index]
    modified_string = filename.replace("_(IaC)_", "", 1)
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]

    
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]

    #print(column_values)
    nb_files = (column_values > 0.5).sum()
    #print(nb_files)


    list_files_from_test_set_until_commitX=extract_files(test_commits, commit_obj, filename)
    #print(list_files_from_test_set_until_commitX)

    missing_files = [file for file in list_files_from_test_set_until_commitX if file not in column_values.index]
    #print(missing_files)


    base_path1 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/"
    base_path2 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"   
    score=0
    scores = []
    for file in missing_files:
      ##print(file)
        modified_file = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', file)
        #print(modified_file)
        full_path1 = os.path.join(base_path1, modified_file)
        full_path2 = os.path.join(base_path2, modified_file)
        #print(full_path1)
        #print(full_path2)
        
        path1_exists = os.path.exists(full_path1)
        path2_exists = os.path.exists(full_path2)
        #print(path1_exists,path2_exists)

        # Determine which path to use based on the file's existence
        if path1_exists:
            #print("path1")
            path_to_use = base_path1
            
        elif path2_exists:
            #print("path2")
            path_to_use = base_path2
        
        else:
            scores.append(0.0)
            continue
        path2 = path_to_use + modified_file
        path1 = path_to_use + modified_string
        #print(path2)
        #print(path1)
        # Attempt to calculate the content score, handling FileNotFoundError
        try:
            score = calcualte_path_score(path1, path2)
            #print(score)
            scores.append(score)
        except FileNotFoundError:
            # Handle the case where the file does not exist
            print(f"One of the files was not found: {path1} or {path2}")
            # Decide on appropriate handling, such as appending a default score or logging
            scores.append(0.0)
    


    count = len([x for x in scores if x > 0.5])
    nb_files=nb_files+count
    #print(nb_files)
    
    return nb_files

In [1888]:
def avg_path_sim_files(filename,commit_obj,csv_hist_path,csv_path_sim, path,train_commits,test_commits):
    avg=0
    std=0
    files_avg=[]


    df = pd.read_csv(csv_hist_path, index_col=0)
    ## process filenae
    matching_files_row = [index for index in df.index if filename in index]
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]

    
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]

    #print(column_values)

    column_values = df.loc[matching_file_row][column_values > 0]
    #print(column_values)

    df1 = pd.read_csv(csv_path_sim, index_col=0)
    matching_files_row1 = [index for index in df1.index if filename in index]
    #print(matching_files_row)
    if not matching_files_row1:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row1 = matching_files_row1[0]

    
    # Get the column values for the matching file
    column_values1 = df1.loc[matching_file_row1]

    #print(column_values1)

    column_values1 = column_values1[column_values1.index.isin(column_values.index)]

    #print(column_values1)


    list_files_from_test_set_until_commitX=extract_files_avg_hist(test_commits, commit_obj, filename)

    #print(list_files_from_test_set_until_commitX)

    missing_files = [file for file in list_files_from_test_set_until_commitX if file not in column_values1.index]
    #print(missing_files)
   

    modified_string = filename.replace("_(IaC)_", "", 1)


    base_path1 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/"
    base_path2 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"   
    score=0
    scores = []
    for file in missing_files:
      ##print(file)
        modified_file = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', file)
        #print(modified_file)
        full_path1 = os.path.join(base_path1, modified_file)
        full_path2 = os.path.join(base_path2, modified_file)
        #print(full_path1)
        #print(full_path2)
        
        path1_exists = os.path.exists(full_path1)
        path2_exists = os.path.exists(full_path2)
        #print(path1_exists,path2_exists)

        # Determine which path to use based on the file's existence
        if path1_exists:
            #print("path1")
            path_to_use = base_path1
            
        elif path2_exists:
            #print("path2")
            path_to_use = base_path2
        
        else:
            scores.append(0.0)
            continue
        path2 = path_to_use + modified_file
        path1 = path_to_use + modified_string
        #print(path2)
        #print(path1)
        # Attempt to calculate the content score, handling FileNotFoundError
        try:
            score = calcualte_path_score(path1, path2)
            #print(score)
            scores.append(score)
        except FileNotFoundError:
            # Handle the case where the file does not exist
            print(f"One of the files was not found: {path1} or {path2}")
            # Decide on appropriate handling, such as appending a default score or logging
            scores.append(0.0)
    

    another_list = [val for val in column_values1.values]

    files_avg=another_list+scores
    #print(files_avg)
    avg=np.mean(files_avg)
    std=np.std(files_avg)
    #print(avg,std)
    return avg, std

In [1889]:
def avg_content_sim_files(filename,commit_obj,csv_hist_path,csv_content_sim, path,train_commits,test_commits):
    avg=0
    std=0
    files_avg=[]
    modified_file_paths=[]
    modified_file_paths1=[]
    #base_path1 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/"

    df = pd.read_csv(csv_hist_path, index_col=0)
    ## process filenae
    modified_string = filename.replace("_(IaC)_", "", 1)
    ## process filenae
    #print(modified_string)
    matching_files_row = [index for index in df.index if modified_string in index]
    #print(matching_files_row)
    if not matching_files_row:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row = matching_files_row[0]

    
    # Get the column values for the matching file
    column_values = df.loc[matching_file_row]

    #print(column_values)

    column_values = df.loc[matching_file_row][column_values > 0]
    #print(column_values)

    for file in column_values.index:
        modified_file_paths.append("/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/" + file)
    
    #modified_file_paths = [print(file) for file in column_values.index]
    #modified_file_paths = [("/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/" + file) for file in column_values.index]

    #modified_file_paths1 = [("/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/" + file) for file in column_values.index]

    #print(modified_file_paths)

    for file in modified_file_paths:
        modified_file_paths1.append(re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', file))
    
    #print(modified_file_paths1)



    df1 = pd.read_csv(csv_content_sim, index_col=0)
    matching_files_row1 = [index for index in df1.index if modified_string in index]
    #print(matching_files_row)
    if not matching_files_row1:
        return "No matching file found."
    
    # Assuming the first match is the desired one
    matching_file_row1 = matching_files_row1[0]

    
    # Get the column values for the matching file
    column_values1 = df1.loc[matching_file_row1]

    #print(column_values1)

    column_values1 = column_values1[column_values1.index.isin(modified_file_paths1)]

    #print(column_values1)


    list_files_from_test_set_until_commitX=extract_files_avg_hist(test_commits, commit_obj, filename)

    #print(list_files_from_test_set_until_commitX)

    missing_files = [file for file in list_files_from_test_set_until_commitX if file not in column_values1.index]
    #print(missing_files) 
   

    modified_string = filename.replace("_(IaC)_", "", 1)


    base_path1 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/"
    base_path2 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"   
    score=0
    scores = []
    for file in missing_files:
      ##print(file)
        modified_file = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', file)
        #print(modified_file)
        full_path1 = os.path.join(base_path1, modified_file)
        full_path2 = os.path.join(base_path2, modified_file)
        path1_exists = os.path.exists(full_path1)
        path2_exists = os.path.exists(full_path2)
        if path1_exists:
            #print("path1")
            path_to_use = base_path1
            
        elif path2_exists:
            #print("path2")
            path_to_use = base_path2
        
        else:
            scores.append(0.0)
            continue
        path2 = path_to_use + modified_file
        path1 = path_to_use + modified_string
        #print(path2)
        #print(path1)
        # Attempt to calculate the content score, handling FileNotFoundError
        try:
            score = calculate_content_score(path1, path2)
            #print(score)
            scores.append(score)
        except FileNotFoundError:
            # Handle the case where the file does not exist
            print(f"One of the files was not found: {path1} or {path2}")
            # Decide on appropriate handling, such as appending a default score or logging
            scores.append(0.0)
    

    another_list = [val for val in column_values1.values]

    files_avg=another_list+scores
    #print(files_avg)
    avg=np.mean(files_avg)
    std=np.std(files_avg)
    #print(avg,std)
    
    return avg, std

In [1890]:
def similar_developers_and_coupling(repo,path,filename,commit_obj):
    modified_string = filename.replace("_(IaC)_", "", 1)
    #repo = Repo("/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"+project)
    if '/' in modified_string:
        parts = modified_string.split('/')
        modified_string = '/'.join(parts[1:])
    authors_file = set()
    co_files=set()
    IaC_co_files=set()
    Other_co_files = set()
    #print(co_files)
    authors_co_files = set()

    commit = repo.commit(commit_obj)
    commit_date = commit.committed_datetime

    commits_touching_file = list(repo.iter_commits(paths=modified_string))
    #print(commits_touching_file)
    for commit in commits_touching_file:
      if commit.committed_datetime < commit_date:  
       
        files_modified = commit.stats.files.keys() # Extracting the keys (file paths) from the dictionary
        author = commit.author
        #print(files_modified)
        for file in files_modified:
            IaC_co_files.add(file)
            #print(file, modified_string)
            if file == modified_string:
                authors_file.add(author.email)
            else:
                co_files.add(file)
                #authors_co_files.add(author.email)

    for file in co_files:
        commits_touching_file = list(repo.iter_commits(paths=file))
        for commit in commits_touching_file:
          if commit.committed_datetime < commit_date:  
            author = commit.author
            authors_co_files.add(author.email)
            for file in files_modified:
                Other_co_files.add(file)

    develoeprs =   len(authors_file)/  len(authors_co_files)  if len(authors_co_files) > 0 else 0
    coupling=  len(IaC_co_files) /    len(Other_co_files) if len(Other_co_files) > 0 else 0

    #intersection = authors_file.intersection(authors_co_files)
    #union = authors_file.union(authors_co_files)

    #inter = IaC_co_files.intersection(Other_co_files)
    
    
    #uni = IaC_co_files.union(Other_co_files)

    # Calculate Jaccard similarity
    #develoeprs = len(intersection) / len(union) if len(union) > 0 else 0
    #print(develoeprs)

    # Calculate Jaccard similarity
    #coupling = len(inter) / len(uni) if len(uni) > 0 else 0

    commits = list(repo.iter_commits(f"{commit_obj}.."))
    authors = set()
    for commit in commits:
        authors.add(commit.author.name)
    ratio_developers = len(authors_file)/len(authors)

    return develoeprs,coupling, ratio_developers

        

In [1891]:
def PoorCode(repo, path, filename, commit_obj):
    # Optimize string modification
    modified_string = filename.replace("_(IaC)_", "", 1)
    if '/' in modified_string:
        modified_string = modified_string.split('/', 1)[1]
    
    authors_file = set()
    co_files = set()
    authors_co_files = set()

    for commit in Repository(path, to_commit=commit_obj).traverse_commits():
        try:
            if any(modified_string in [modified_file.new_path, modified_file.old_path] for modified_file in commit.modified_files):
                for modified_file in commit.modified_files:
                    co_files.add(modified_file.new_path)
                    author = commit.author
                    authors_file.add(author.email)
                    
                    #print(author.email)
        except ValueError as e:
            if 'could not be resolved' in str(e) and 'missing' in str(e):
                print(f"Skipping commit {e.args[0]} as it is missing.")
                continue
            else:
                raise
   
    for file in co_files:
        for commit in Repository(path, to_commit=commit_obj).traverse_commits():
            try:
                for modified_file in commit.modified_files:
                    if modified_file.new_path == file or modified_file.old_path == file:
                        authors_co_files.add(commit.author.email)
            except ValueError as e:
                if 'could not be resolved' in str(e) and 'missing' in str(e):
                    print(f"Skipping commit {e.args[0]} as it is missing.")
                    continue
                else:
                    raise
        intersection = authors_file.intersection(authors_co_files)
        union = authors_file.union(authors_co_files)
        
        # Calculate Jaccard similarity
        jaccard_similarity = len(intersection) / len(union)
        #print(jaccard_similarity)
        Union_developers.append(jaccard_similarity)

    percentage_file_to_co_files = np.mean(Union_developers)
    #print(percentage_file_to_co_files)  
    return percentage_file_to_co_files

In [1892]:
def files_coupling(repo,path,filename,commit_obj):
   
    modified_string = filename.replace("_(IaC)_", "", 1)
    #repo = Repo("/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"+project)
    if '/' in modified_string:
        parts = modified_string.split('/')
        modified_string = '/'.join(parts[1:])
    IaC_co_files=set()
    Other_co_files = set()

    commits_touching_file = list(repo.iter_commits(paths=modified_string))
    #print(commits_touching_file)
    for commit in commits_touching_file:
       
        files_modified = commit.stats.files.keys() # Extracting the keys (file paths) from the dictionary
        
        for file in files_modified:
            
                IaC_co_files.add(file)
                #authors_co_files.add(author.email)

    for file in IaC_co_files:
        commits_touching_file = list(repo.iter_commits(paths=file))
        for commit in commits_touching_file:
            files_modified = commit.stats.files.keys()
            for file in files_modified:
                Other_co_files.add(file)

    intersection = IaC_co_files.intersection(Other_co_files)
    union = IaC_co_files.union(Other_co_files)

    # Calculate Jaccard similarity
    jaccard_similarity = len(intersection) / len(union)
    print(jaccard_similarity)
    return jaccard_similarity 

In [1893]:


def History_Graph_Metrics(filename,commit_obj,path,csv_hist_path,test_commits):

    file_path = csv_hist_path
    df = pd.read_csv(file_path, index_col=0)

    # Create an adjacency matrix from the DataFrame
    adj_matrix = df.values
    nodes = list(df.index)

    # Get the index of file_x in the nodes list
    file_x_index = nodes.index(filename)

    New_files=extract_files_avg_hist(test_commits, commit_obj, filename)

    
    ########### ADD new files from the test commits before the current test commit
    # Iterate over the files in files_set
    # Get the count of each file in files_set
    file_count = {}
    for file in New_files:
        if file in file_count:
            file_count[file] += 1
        else:
            file_count[file] = 1

    # Iterate over the files in files_set
    for file, count in file_count.items():
        # Check if the file is already in the nodes list
        if file not in nodes:
            # If not, add the file to the nodes list and the adjacency matrix
            nodes.append(file)
            adj_matrix = np.pad(adj_matrix, ((0, 1), (0, 1)), 'constant')
            adj_matrix[file_x_index, -1] = count
            adj_matrix[-1, file_x_index] = count
        else:
            # If the file is already in the nodes list, increment its value in the adjacency matrix
            file_index = nodes.index(file)
            adj_matrix[file_x_index, file_index] += count
            adj_matrix[file_index, file_x_index] += count

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes and edges to the graph based on the adjacency matrix
    for i, row in enumerate(adj_matrix):
        for j, value in enumerate(row):
            if value > 0:
                G.add_edge(nodes[i], nodes[j], weight=value)
    def create_subgraph(G, start_node):
        visited = set()
        queue = [start_node]

        while queue:
            node = queue.pop(0)
            if node not in visited:
                visited.add(node)
                queue.extend(n for n in G[node] if n not in visited)

        subgraph = G.subgraph(visited)
        return subgraph
    # Specify the node to visualize
    node_to_visualize = filename

    # Map unique numbers to the files
    #file_number_mapping = {file: i for i, file in enumerate(df.index)}

    # Check if the node exists in the graph
    if node_to_visualize in G.nodes():
        #subgraph = create_subgraph(G, node_to_visualize)
        #neighbors = list(G[node_to_visualize])
        # Degree Centrality
        #num_edges = G.degree(node_to_visualize)
        degree_centrality = nx.degree_centrality(G)
        #degree_centrality_filtered = {node: degree_centrality[node] for node in neighbors}
        node_degree_centrality = degree_centrality[node_to_visualize]
        #Avg_Degree_centrality = statistics.mean(degree_centrality_filtered.values())

        # Standard Deviation of Degree Centrality
        if len(degree_centrality) > 1:
            std_dev_degree_centrality = statistics.stdev(degree_centrality.values())
        else:
            std_dev_degree_centrality = 0
        #print(degree_centrality_filtered.values())

        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        #betweenness_centrality_filtered = {node: betweenness_centrality[node] for node in neighbors}
        node_betweenness_centrality = betweenness_centrality[node_to_visualize]
        #Avg_betweenness_centrality = statistics.mean(betweenness_centrality_filtered.values())

        # Closeness Centrality
        closeness_centrality = nx.closeness_centrality(G)
        #closeness_centrality_filtered = {node: closeness_centrality[node] for node in neighbors}
        node_closeness_centrality = closeness_centrality[node_to_visualize]
        #Avg_closeness_centrality = statistics.mean(closeness_centrality_filtered.values())

        # Network Density
        #subgraph = G.subgraph(neighbors)
        network_density = nx.density(G)

        #print(f"Standard Deviation of Degree Centrality for {node_to_visualize}: {std_dev_degree_centrality}")
        ##print("####################################################################################################")
        #print(f"Network Density for {node_to_visualize}: {network_density}")
        #print("####################################################################################################")
        #print(f"Degree Centrality for {node_to_visualize}:{node_degree_centrality}")
        #print("####################################################################################################")
        #print(f"Closeness Centrality for {node_to_visualize}:{node_closeness_centrality}")
        #print("####################################################################################################")
        #print(f"Betweenness Centrality for {node_to_visualize}:{node_betweenness_centrality}")
        #print("####################################################################################################") 
    #print(std_dev_degree_centrality)
    #print(Avg_betweenness_centrality)
    #print(Avg_closeness_centrality)
    #print(network_density)
    #print(Avg_Degree_centrality)
    return std_dev_degree_centrality,network_density,node_degree_centrality, node_betweenness_centrality,node_closeness_centrality



In [1894]:
def calcul_similarity(IaCfile, Newfile):
    base_path1 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/"
    base_path2 = "/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"   
    score=0
    scores = []
    
    modified_file = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', IaCfile)
    #print(modified_file)
    full_path1 = os.path.join(base_path1, modified_file)
    full_path2 = os.path.join(base_path2, modified_file)
    path1_exists = os.path.exists(full_path1)
    path2_exists = os.path.exists(full_path2)
    if path1_exists:
        #print("path1")
        path_to_use = base_path1
        
    elif path2_exists:
        #print("path2")
        path_to_use = base_path2
    
    else:
        scores.append(0.0)
        
    path2 = path_to_use + modified_file
    path1 = path_to_use + Newfile
    #print(path2)
    #print(path1)
    # Attempt to calculate the content score, handling FileNotFoundError
    try:
        score = calculate_content_score(path1, path2)
        #print(score)
        #scores.append(score)
    except FileNotFoundError:
        # Handle the case where the file does not exist
        print(f"One of the files was not found: {path1} or {path2}")
        score=0.0

    return score



In [1895]:


def ContentSim_Graph_Metrics(filename,commit_obj,path,csv_hist_path,csv_content_sim,test_commits):
    std_dev_degree_centrality=0
    file_path = csv_content_sim
    df = pd.read_csv(file_path, index_col=0)

    # Create an adjacency matrix from the DataFrame
    adj_matrix = df.values
    nodes = list(df.index)

    # Get the index of file_x in the nodes list
    modified_file = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', filename)
    base_path1='/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/'
    full_path1 = os.path.join(base_path1, modified_file)
    
    file_x_index = nodes.index(full_path1)

    New_files=extract_files_avg_hist(test_commits, commit_obj, filename)

    
    ########### ADD new files from the test commits before the current test commit
    # Iterate over the files in files_set
    # Get the count of each file in files_set
    file_count = {}
    for file in New_files:
        if file in file_count:
            file_count[file] += 1
        else:
            file_count[file] = 1

    for Testfile in New_files:
        # Iterate over the files in files_set
        for file, count in file_count.items():
            # Check if the file is already in the nodes list
            if file not in nodes:
                count=calcul_similarity(filename, Testfile)
                # If not, add the file to the nodes list and the adjacency matrix
                nodes.append(file)
                adj_matrix = np.pad(adj_matrix, ((0, 1), (0, 1)), 'constant')
                adj_matrix[file_x_index, -1] = count
                adj_matrix[-1, file_x_index] = count
            #else:
                # If the file is already in the nodes list, increment its value in the adjacency matrix
             #   file_index = nodes.index(file)
              #  adj_matrix[file_x_index, file_index] += count
               # adj_matrix[file_index, file_x_index] += count

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes and edges to the graph based on the adjacency matrix
    for i, row in enumerate(adj_matrix):
        for j, value in enumerate(row):
            if value > 0:
                G.add_edge(nodes[i], nodes[j], weight=value)
    def create_subgraph(G, start_node):
        visited = set()
        queue = [start_node]

        while queue:
            node = queue.pop(0)
            if node not in visited:
                visited.add(node)
                queue.extend(n for n in G[node] if n not in visited)

        subgraph = G.subgraph(visited)
        return subgraph
    # Specify the node to visualize
    node_to_visualize = full_path1

    # Map unique numbers to the files
    #file_number_mapping = {file: i for i, file in enumerate(df.index)}

    # Check if the node exists in the graph
    if node_to_visualize in G.nodes():
        #subgraph = create_subgraph(G, node_to_visualize)
        #neighbors = list(G[node_to_visualize])
        # Degree Centrality
        #num_edges = G.degree(node_to_visualize)
        degree_centrality = nx.degree_centrality(G)
        #degree_centrality_filtered = {node: degree_centrality[node] for node in neighbors}
        node_degree_centrality = degree_centrality[node_to_visualize]
        #Avg_Degree_centrality = statistics.mean(degree_centrality_filtered.values())

        # Standard Deviation of Degree Centrality
        if len(degree_centrality) > 1:
            std_dev_degree_centrality = statistics.stdev(degree_centrality.values())
        else:
            std_dev_degree_centrality = 0
        #print(degree_centrality_filtered.values())

        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        #betweenness_centrality_filtered = {node: betweenness_centrality[node] for node in neighbors}
        node_betweenness_centrality = betweenness_centrality[node_to_visualize]
        #Avg_betweenness_centrality = statistics.mean(betweenness_centrality_filtered.values())

        # Closeness Centrality
        closeness_centrality = nx.closeness_centrality(G)
        #closeness_centrality_filtered = {node: closeness_centrality[node] for node in neighbors}
        node_closeness_centrality = closeness_centrality[node_to_visualize]
        #Avg_closeness_centrality = statistics.mean(closeness_centrality_filtered.values())

        # Network Density
        #subgraph = G.subgraph(neighbors)
        network_density = nx.density(G)

        #print(f"Standard Deviation of Degree Centrality for {node_to_visualize}: {std_dev_degree_centrality}")
        ##print("####################################################################################################")
        #print(f"Network Density for {node_to_visualize}: {network_density}")
        #print("####################################################################################################")
        #print(f"Degree Centrality for {node_to_visualize}:{node_degree_centrality}")
        #print("####################################################################################################")
        #print(f"Closeness Centrality for {node_to_visualize}:{node_closeness_centrality}")
        #print("####################################################################################################")
        #print(f"Betweenness Centrality for {node_to_visualize}:{node_betweenness_centrality}")
        #print("####################################################################################################") 
    #print(std_dev_degree_centrality)
    #print(Avg_betweenness_centrality)
    #print(Avg_closeness_centrality)
    #print(network_density)
    #print(Avg_Degree_centrality)
    return std_dev_degree_centrality,network_density,node_degree_centrality, node_betweenness_centrality,node_closeness_centrality



In [1896]:


def PathSim_Graph_Metrics(filename,commit_obj,path,csv_hist_path,csv_path_sim,test_commits):
    std_dev_degree_centrality=0
    file_path = csv_path_sim
    df = pd.read_csv(file_path, index_col=0)

    # Create an adjacency matrix from the DataFrame
    adj_matrix = df.values
    nodes = list(df.index)

    # Get the index of file_x in the nodes list
    #modified_file = re.sub(r'(_\(IaC\)_|_\(Other\)_)', '', filename)
    #base_path1='/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/'
    #full_path1 = os.path.join(base_path1, modified_file)
    
    file_x_index = nodes.index(filename)

    New_files=extract_files_avg_hist(test_commits, commit_obj, filename)

    
    ########### ADD new files from the test commits before the current test commit
    # Iterate over the files in files_set
    # Get the count of each file in files_set
    file_count = {}
    for file in New_files:
        if file in file_count:
            file_count[file] += 1
        else:
            file_count[file] = 1

    for Testfile in New_files:
        # Iterate over the files in files_set
        for file, count in file_count.items():
            # Check if the file is already in the nodes list
            if file not in nodes:
                count=calcualte_path_score(filename, Testfile)
                # If not, add the file to the nodes list and the adjacency matrix
                nodes.append(file)
                adj_matrix = np.pad(adj_matrix, ((0, 1), (0, 1)), 'constant')
                adj_matrix[file_x_index, -1] = count
                adj_matrix[-1, file_x_index] = count
            #else:
                # If the file is already in the nodes list, increment its value in the adjacency matrix
             #   file_index = nodes.index(file)
              #  adj_matrix[file_x_index, file_index] += count
               # adj_matrix[file_index, file_x_index] += count

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes and edges to the graph based on the adjacency matrix
    for i, row in enumerate(adj_matrix):
        for j, value in enumerate(row):
            if value > 0:
                G.add_edge(nodes[i], nodes[j], weight=value)
    def create_subgraph(G, start_node):
        visited = set()
        queue = [start_node]

        while queue:
            node = queue.pop(0)
            if node not in visited:
                visited.add(node)
                queue.extend(n for n in G[node] if n not in visited)

        subgraph = G.subgraph(visited)
        return subgraph
    # Specify the node to visualize
    node_to_visualize = filename

    # Map unique numbers to the files
    #file_number_mapping = {file: i for i, file in enumerate(df.index)}

    # Check if the node exists in the graph
    if node_to_visualize in G.nodes():
        #subgraph = create_subgraph(G, node_to_visualize)
        #neighbors = list(G[node_to_visualize])
        # Degree Centrality
        #num_edges = G.degree(node_to_visualize)
        degree_centrality = nx.degree_centrality(G)
        #degree_centrality_filtered = {node: degree_centrality[node] for node in neighbors}
        node_degree_centrality = degree_centrality[node_to_visualize]
        #Avg_Degree_centrality = statistics.mean(degree_centrality_filtered.values())

        # Standard Deviation of Degree Centrality
        if len(degree_centrality) > 1:
            std_dev_degree_centrality = statistics.stdev(degree_centrality.values())
        else:
            std_dev_degree_centrality = 0
        #print(degree_centrality_filtered.values())

        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        #betweenness_centrality_filtered = {node: betweenness_centrality[node] for node in neighbors}
        node_betweenness_centrality = betweenness_centrality[node_to_visualize]
        #Avg_betweenness_centrality = statistics.mean(betweenness_centrality_filtered.values())

        # Closeness Centrality
        closeness_centrality = nx.closeness_centrality(G)
        #closeness_centrality_filtered = {node: closeness_centrality[node] for node in neighbors}
        node_closeness_centrality = closeness_centrality[node_to_visualize]
        #Avg_closeness_centrality = statistics.mean(closeness_centrality_filtered.values())

        # Network Density
        #subgraph = G.subgraph(neighbors)
        network_density = nx.density(G)

        #print(f"Standard Deviation of Degree Centrality for {node_to_visualize}: {std_dev_degree_centrality}")
        ##print("####################################################################################################")
        #print(f"Network Density for {node_to_visualize}: {network_density}")
        #print("####################################################################################################")
        #print(f"Degree Centrality for {node_to_visualize}:{node_degree_centrality}")
        #print("####################################################################################################")
        #print(f"Closeness Centrality for {node_to_visualize}:{node_closeness_centrality}")
        #print("####################################################################################################")
        #print(f"Betweenness Centrality for {node_to_visualize}:{node_betweenness_centrality}")
        #print("####################################################################################################") 
    #print(std_dev_degree_centrality)
    #print(Avg_betweenness_centrality)
    #print(Avg_closeness_centrality)
    #print(network_density)
    #print(Avg_Degree_centrality)
    return std_dev_degree_centrality,network_density,node_degree_centrality, node_betweenness_centrality,node_closeness_centrality



In [1897]:

def process_data(GA_precision, PSO_precision,RS_precision,SA_precision,HIST_precision,SIM_precision,commits,filename,co_modified_files,counter,csv_hist_path,csv_path_sim,csv_content_sim,test_commits,train_commits,project):
    print(filename)
    
    
    
    instances=[]
    feature_file_loc=[]
    feature_iac_file_churn=[]
    feature_file_halstead_complexity=[]
    feature_file_cyclomatic_complexity=[]
    feature_file_dependency=[]
    feature_num_ImportedPlaybooks=[]
    feature_num_NumImportedRoles=[]
    feature_num_NumImportedTasks=[]
    feature_num_NumIncludedRoles=[]
    feature_num_NumIncludedVars=[]
    feature_num_NumIncludes=[]
    feature_num_NumIncludedTasks=[]
    feature_nb_files=[]
    feature_nb_iac_files=[]
    feature_nb_noniac_files=[]
    feature_avg_files_per_commit=[]
    feature_avg_iac_files_per_commit=[]
    feature_avg_noniac_files_per_commit=[]
    feature_file_avg_history=[]
    feature_file_std_history=[]
    feature_nb_content_sim_files=[]
    feature_nb_path_sim_files=[]
    feature_avg_path_sim_files=[]
    feature_std_path_sim_files=[]
    feature_avg_content_sim_files=[]
    feature_std_content_sim_files=[]
    feature_Ratio_out_of_cofiles_developers=[]
    feature_Ratio_out_of_all_developers=[]
    feature_files_coupling=[]
    feature_files_nb_parameters=[]
    feature_files_nb_variables=[]
    feature_files_avg_play_size=[]
    feature_files_avg_task_size=[]
    feature_files_num_plays=[]
    feature_files_num_tasks=[]

    feature_files_num_commands=[]
    feature_files_num_blocks=[]
    feature_files_num_Distinctmodules=[]
    feature_files_num_ExternalModules=[]
    feature_files_num_FactModules=[]
    feature_files_num_Paths=[]
    feature_files_num_Roles=[]
    feature_files_num_DeprecatedModules=[]

    feature_files_num_normalized_resources=[]


    feature_Graph_std_degree_centrality=[]
    feature_Graph_network_density=[]
    feature_Graph_Avg_Degree_centrality=[]
    feature_Graph_Avg_betweenness_centrality=[]
    feature_Graph_Avg_closeness_centrality=[]

    feature_Graph_ContentSimilarity_std_degree_centrality=[]
    feature_Graph_ContentSimilarity_network_density=[]
    feature_Graph_ContentSimilarity_Avg_Degree_centrality=[]
    feature_Graph_ContentSimilarity_Avg_betweenness_centrality=[]
    feature_Graph_ContentSimilarity_Avg_closeness_centrality=[]


    feature_Graph_PathSimilarity_std_degree_centrality=[]
    feature_Graph_PathSimilarity_network_density=[]
    feature_Graph_PathSimilarity_Avg_Degree_centrality=[]
    feature_Graph_PathSimilarity_Avg_betweenness_centrality=[]
    feature_Graph_PathSimilarity_Avg_closeness_centrality=[]


    
    
    for i in range(len(GA_precision)):
        instances.append(filename)
  
    commit_hashes = []
    for commit_str in commits:
        match = re.match(r'^.*?::([0-9a-f]{40})$', commit_str)
        if match:
            commit_hash = match.group(1)
            commit_hashes.append(commit_hash)
        else:
            print(f"Invalid commit string: {commit_str}")

    
                #commit = hashlib.sha1(commits.encode()).hexdigest()
    repo = Repo("/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"+project)
    path="/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits_LastVersion/"+project
    #print(commit_hashes)
    for commit in commit_hashes:
            commit_obj = repo.commit(commit)
            print(commit_obj)
            
            churn_iac_file=file_churn(filename,commit_obj,path) 
            #print(churn_iac_file) 
            feature_iac_file_churn.append(churn_iac_file)

            loc_iac_file=file_loc(filename,commit_obj,path) 
            feature_file_loc.append(loc_iac_file)

            nb_resources=file_ansible_resources(filename,commit_obj,path) 
            feature_files_num_normalized_resources.append(round(nb_resources,2))

            
            nbparam=ansible_num_parameters(filename,commit_obj,path) 
            feature_files_nb_parameters.append(nbparam)

            nbvars=ansible_num_variables(filename,commit_obj,path) 
            feature_files_nb_variables.append(nbvars)


            

            avg_play_size,avg_task_size,num_plays,num_tasks,num_commands,num_blocks,num_Distinctmodules,num_ExternalModules,num_FactModules,num_Paths,num_Roles,num_DeprecatedModules=ansible_quality(filename,commit_obj,path)
            feature_files_avg_play_size.append(avg_play_size)
            feature_files_avg_task_size.append(avg_task_size)
            feature_files_num_plays.append(num_plays)
            feature_files_num_tasks.append(num_tasks)
            feature_files_num_commands.append(num_commands)
            feature_files_num_blocks.append(num_blocks)
            feature_files_num_Distinctmodules.append(num_Distinctmodules)
            feature_files_num_ExternalModules.append(num_ExternalModules)
            feature_files_num_FactModules.append(num_FactModules)
            feature_files_num_Paths.append(num_Paths)
            feature_files_num_Roles.append(num_Roles)
            feature_files_num_DeprecatedModules.append(num_DeprecatedModules)



            file_depen,num_ImportedPlaybooks,num_NumImportedRoles,num_NumImportedTasks,num_NumIncludedRoles,num_NumIncludedVars,num_NumIncludes,num_NumIncludedTasks= iac_dependency(filename,commit_obj,path) 
            feature_file_dependency.append(file_depen) 
            feature_num_ImportedPlaybooks.append(num_ImportedPlaybooks)
            feature_num_NumImportedRoles.append(num_NumImportedRoles)
            feature_num_NumImportedTasks.append(num_NumImportedTasks)
            feature_num_NumIncludedRoles.append(num_NumIncludedRoles)
            feature_num_NumIncludedVars.append(num_NumIncludedVars)
            feature_num_NumIncludes.append(num_NumIncludes)
            feature_num_NumIncludedTasks.append(num_NumIncludedTasks)
 
            std_dev_degree_centrality,network_density,Avg_Degree_centrality, Avg_betweenness_centrality,Avg_closeness_centrality=History_Graph_Metrics(filename,commit_obj,path,csv_hist_path,test_commits)
            feature_Graph_std_degree_centrality.append(round(std_dev_degree_centrality,2))
            feature_Graph_network_density.append(round(network_density,2))
            feature_Graph_Avg_Degree_centrality.append(round(Avg_Degree_centrality,2))
            feature_Graph_Avg_betweenness_centrality.append(round(Avg_betweenness_centrality,2))
            feature_Graph_Avg_closeness_centrality.append(round(Avg_closeness_centrality,2))



            std_dev_degree_centrality,network_density,Avg_Degree_centrality, Avg_betweenness_centrality,Avg_closeness_centrality=ContentSim_Graph_Metrics(filename,commit_obj,path,csv_hist_path,csv_content_sim,test_commits)
            feature_Graph_ContentSimilarity_std_degree_centrality.append(round(std_dev_degree_centrality,2))
            feature_Graph_ContentSimilarity_network_density.append(round(network_density,2))
            feature_Graph_ContentSimilarity_Avg_Degree_centrality.append(round(Avg_Degree_centrality,2))
            feature_Graph_ContentSimilarity_Avg_betweenness_centrality.append(round(Avg_betweenness_centrality,2))
            feature_Graph_ContentSimilarity_Avg_closeness_centrality.append(round(Avg_closeness_centrality,2))


            std_dev_degree_centrality,network_density,Avg_Degree_centrality, Avg_betweenness_centrality,Avg_closeness_centrality=PathSim_Graph_Metrics(filename,commit_obj,path,csv_hist_path,csv_path_sim,test_commits)
            feature_Graph_PathSimilarity_std_degree_centrality.append(round(std_dev_degree_centrality,2))
            feature_Graph_PathSimilarity_network_density.append(round(network_density,2))
            feature_Graph_PathSimilarity_Avg_Degree_centrality.append(round(Avg_Degree_centrality,2))
            feature_Graph_PathSimilarity_Avg_betweenness_centrality.append(round(Avg_betweenness_centrality,2))
            feature_Graph_PathSimilarity_Avg_closeness_centrality.append(round(Avg_closeness_centrality,2))


            
            nbfiles=number_files(filename,commit_obj,csv_hist_path, path,test_commits)
            feature_nb_files.append(nbfiles)  

            nb_iac=number_iac_files(filename,commit_obj,csv_hist_path, path,test_commits)
            feature_nb_iac_files.append(nb_iac)

            nb_noniac=number_noniac_files(filename,commit_obj,csv_hist_path, path,test_commits)
            feature_nb_noniac_files.append(nb_noniac)

            avg_files=Avg_files(filename,commit_obj,csv_hist_path, path,train_commits,test_commits)
            feature_avg_files_per_commit.append(round(avg_files,2))

            avg_iac_files=Avg_iac_files(filename,commit_obj,csv_hist_path, path,train_commits,test_commits)
            feature_avg_iac_files_per_commit.append(round(avg_iac_files,2))

            avg_noniac_files=Avg_noniac_files(filename,commit_obj,csv_hist_path, path,train_commits,test_commits)
            feature_avg_noniac_files_per_commit.append(round(avg_noniac_files,2))


            #avg_dependency= iac_avg_dependency(filename,commit_obj,repo,path,train_commits,test_commits) 
            #feature_file_avg_dependency.append(round(avg_dependency,2))


            avg_hist,std_hist=file_avg_history(filename,commit_obj,csv_hist_path, path,test_commits)
            feature_file_avg_history.append(round(avg_hist,2))
            feature_file_std_history.append(round(std_hist,2))

            nb_files=nb_content_sim_files(filename,commit_obj,csv_content_sim, path,train_commits,test_commits)
            feature_nb_content_sim_files.append(nb_files)


            nb_files=nb_path_sim_files(filename,commit_obj,csv_path_sim, path,train_commits,test_commits)
            feature_nb_path_sim_files.append(nb_files)

            avg_path_files, std_path_files=avg_path_sim_files(filename,commit_obj,csv_hist_path,csv_path_sim, path,train_commits,test_commits)
            feature_avg_path_sim_files.append(round(avg_path_files,2))
            feature_std_path_sim_files.append(round(std_path_files,2))



            avg_content_files, std_content_files=avg_content_sim_files(filename,commit_obj,csv_hist_path,csv_content_sim,path,train_commits,test_commits)
            feature_avg_content_sim_files.append(round(avg_content_files,2))
            feature_std_content_sim_files.append(round(std_content_files,2))

            
            similar_devs,coupling,ratio_developers=similar_developers_and_coupling(repo,path,filename,commit_obj)
            feature_Ratio_out_of_cofiles_developers.append(round(similar_devs,2))
            feature_files_coupling.append(round(coupling,2))
            feature_Ratio_out_of_all_developers.append(round(ratio_developers,2))




  


    seed=save_data(GA_precision, PSO_precision,RS_precision,SA_precision,HIST_precision,SIM_precision,commits,instances,feature_file_loc,feature_iac_file_churn,feature_file_dependency,feature_nb_files,feature_nb_iac_files,feature_nb_noniac_files,feature_avg_files_per_commit,feature_avg_iac_files_per_commit,feature_avg_noniac_files_per_commit,feature_file_avg_history,feature_file_std_history,feature_nb_content_sim_files,feature_nb_path_sim_files,feature_avg_path_sim_files,feature_std_path_sim_files,feature_avg_content_sim_files,feature_std_content_sim_files,feature_Ratio_out_of_cofiles_developers,feature_Ratio_out_of_all_developers,feature_files_coupling,feature_files_nb_parameters,feature_files_nb_variables,feature_files_avg_play_size,feature_files_avg_task_size,feature_files_num_plays,feature_files_num_tasks,feature_files_num_commands,feature_files_num_blocks,feature_files_num_Distinctmodules,feature_files_num_ExternalModules,feature_files_num_FactModules,feature_files_num_Paths,feature_files_num_Roles,feature_files_num_DeprecatedModules,
                   feature_num_NumImportedRoles,feature_num_ImportedPlaybooks,feature_num_NumImportedTasks,feature_num_NumIncludedRoles,feature_num_NumIncludedVars,feature_num_NumIncludes,feature_num_NumIncludedTasks,feature_files_num_normalized_resources,
                   feature_Graph_std_degree_centrality,feature_Graph_network_density,feature_Graph_Avg_Degree_centrality,feature_Graph_Avg_betweenness_centrality,feature_Graph_Avg_closeness_centrality,
                    feature_Graph_ContentSimilarity_std_degree_centrality,feature_Graph_ContentSimilarity_network_density,feature_Graph_ContentSimilarity_Avg_Degree_centrality,feature_Graph_ContentSimilarity_Avg_betweenness_centrality,feature_Graph_ContentSimilarity_Avg_closeness_centrality,
                    feature_Graph_PathSimilarity_std_degree_centrality,feature_Graph_PathSimilarity_network_density,feature_Graph_PathSimilarity_Avg_Degree_centrality,feature_Graph_PathSimilarity_Avg_betweenness_centrality,feature_Graph_PathSimilarity_Avg_closeness_centrality,counter)     
    return seed            

In [1898]:
def process_number(number):
    # Check if the number is an integer or effectively an integer (e.g., 1.0)
    if number==1:
       return float(number)
    #int(number) == number:
        #return int(number*100)  # Return the integer part
    else:
        # For numbers with a significant decimal part, extract the first two digits of the decimal part
        decimal_part = str(number).split('.')[1][:2]  # Take first two digits of the decimal part
        return float(decimal_part) if decimal_part else 0  # Return as integer, or 0 if not present


In [1899]:

#algos= ['Dynamic', 'PSO_FINAL', 'RS_FINAL','SA_FINAL']
# 'NSGAPOP','NSGANPPareto','NSGANPFixed'
if __name__ == '__main__':
  seed = 0
  for project in projects:
    csv_hist_path="/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_7/Projects/"+project+"/Matrices_90_10/co_change_matrix.csv"
    csv_path_sim="/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_7/Projects/"+project+"/Matrices_90_10/path_sim_matrix.csv"
    csv_content_sim ="/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_7/Projects/"+project+"/Matrices_90_10/content_sim_matrix.csv"
    test_commits="/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_7/Projects/"+project+"/_10_test_Of_Commits_hash.csv"
    train_commits="/Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_7/Projects/"+project+"/_90_train_Of_Commits_hash.csv"
    for Algo in algos:
      for TopK in tops:

        for metric in median_indexes:

            folder_path = path + project + '/' + Algo + '/' + str(
                TopK)

            # Initialize lists to store the filtered values
            GA_precision = []
            PSO_precision = []
            RS_precision=[]
            SA_precision=[]
            HIST_precision=[]
            SIM_precision=[]
            commits = []
            co_modified_files = []
            success = []
            # Iterate through each folder in the root folder

            for folder_name in os.listdir(folder_path):
                #print(folder_name)

                # Check if the folder name contains the string "FilesPlots"
                if metric in folder_name and os.path.isdir(folder_path):
                    folder_path_metric = os.path.join(folder_path, folder_name)
                    #print(folder_path_metric)
                    for folder_name in os.listdir(folder_path_metric):
                        #print(folder_name)

                        #         # Check if the folder name contains the string "FilesPlots"
                        if 'Results_Median' in folder_name and os.path.isdir(folder_path_metric):
                            #print(folder_name)
                            folder_path = os.path.join(folder_path_metric, folder_name)
                            #print(folder_path)
                            for filename in os.listdir(folder_path):
                                #print(filename)
                                if filename.endswith('.csv'):
                                    #GA
                                     
                                    file_path = os.path.join(folder_path, filename)
                                    with open(file_path, 'r') as csv_file:
                                        #print(file_path)
                                        reader = csv.DictReader(csv_file)
                                        precision_sum = 0
                                        for row in reader:
                                            try:
                                                prec = float(row['PRECISION_average'])
                                                #print(prec)
                                                comm = str(row['Commits_hash'])
                                                files = str(row['Co_modified_files'])
                                                #print(files)
                                                if prec >= 0.0:
                                                    precision_sum = prec
                                                    #print(prec)
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    #print(decimal_as_int)
                                                    #print(commits)
                                                    #print(co_modified_files)
                                                    GA_precision.append(round(precision_sum,2))
                                                    source=project+"::"+comm
                                                    commits.append(source)
                                                    co_modified_files.append(files)
                                                    success.append(1)
                                                else:
                                                    precision_sum = prec
                                                    #print(precision_sum)
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    GA_precision.append(round(precision_sum,2))
                                                    source=project+"::"+comm
                                                    commits.append(source)
                                                    #commits.append(comm)
                                                    co_modified_files.append(files)
                                                    #print(GA_precision)
                                                    #print(commits)
                                                    success.append(0)    
                                            except (ValueError, KeyError):
                                                continue 

                                    #print(co_modified_files)       
                                    #PSO
                                    folder_path_sim = path + project + '/' + "GA_SIM" + '/' + str(TopK)
                                    folder_path_metric_sim = os.path.join(folder_path_sim, metric)
                                    #print(folder_path_metric)
                                    folder_path_sim = os.path.join(folder_path_metric_sim, folder_name)        
                                    file_path_sim = os.path.join(folder_path_sim, filename)
                                    #print(file_path_sim)
                                    with open(file_path_sim, 'r') as csv_file:
                                        #print(file_path3)
                                        reader = csv.DictReader(csv_file)
                                        precision_sum = 0
                                        for row in reader:
                                            try:
                                                prec = float(row['PRECISION_average'])
                                                if prec >= 0.0:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    SIM_precision.append(round(precision_sum,2))
                                                else:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    SIM_precision.append(round(precision_sum,2))    
                                            except (ValueError, KeyError):
                                                continue 


                                    #RS
                                    folder_path_hist = path + project + '/' + "GA_HIST" + '/' + str(TopK)
                                    #print(folder_path_hist)
                                    folder_path_metric_hist = os.path.join(folder_path_hist, metric)
                                    #print(folder_path_metric_hist)
                                    folder_path_hist = os.path.join(folder_path_metric_hist, folder_name)    
                                    #print(folder_path_hist)   
                                    #print(filename)  
                                    file_path_hist = os.path.join(folder_path_hist, filename)
                                    #print(file_path_hist)
                                    with open(file_path_hist, 'r') as csv_file:
                                        #print(file_path_RS)
                                        reader = csv.DictReader(csv_file)
                                        precision_sum = 0
                                        for row in reader:
                                            try:
                                                #filename=str(row['IaC_file'])
                                                #print(filename)
                                                prec = float(row['PRECISION_average'])
                                                if prec >= 0.0:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    HIST_precision.append(round(precision_sum,2))
                                                else:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    HIST_precision.append(round(precision_sum,2))    
                                            except (ValueError, KeyError):
                                                continue                                         
                                    
                                    #SA
                                    folder_path_SA = path + project + '/' + "SA_FINAL" + '/' + str(TopK)
                                    #print(folder_path_SA)
                                    folder_path_metric_SA = os.path.join(folder_path_SA, metric)
                                    #print(folder_path_metric_SA)
                                    folder_path_SA = os.path.join(folder_path_metric_SA, folder_name)  
                                    #print(folder_path_SA)   
                                    #print(filename)   
                                    file_path_SA = os.path.join(folder_path_SA, filename)
                                    #print(file_path_SA)
                                    with open(file_path_SA, 'r') as csv_file:
                                        #print(file_path_SA)
                                        reader = csv.DictReader(csv_file)
                                        precision_sum = 0
                                        for row in reader:
                                            try:
                                                prec = float(row['PRECISION_average'])
                                                if prec >= 0.0:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    SA_precision.append(round(precision_sum,2))
                                                else:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    SA_precision.append(round(precision_sum,2))    
                                            except (ValueError, KeyError):
                                                continue       


                                    folder_path1 = path + project + '/' + "PSO_FINAL" + '/' + str(TopK)
                                    folder_path_metric1 = os.path.join(folder_path1, metric)
                                    #print(folder_path_metric)
                                    folder_path2 = os.path.join(folder_path_metric1, folder_name)        
                                    file_path3 = os.path.join(folder_path2, filename)
                                    with open(file_path3, 'r') as csv_file:
                                        #print(file_path3)
                                        reader = csv.DictReader(csv_file)
                                        precision_sum = 0
                                        for row in reader:
                                            try:
                                                prec = float(row['PRECISION_average'])
                                                if prec >= 0.0:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    PSO_precision.append(round(precision_sum,2))
                                                else:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    PSO_precision.append(round(precision_sum,2))    
                                            except (ValueError, KeyError):
                                                continue



                                    
                                    folder_path_RS = path + project + '/' + "RS_FINAL" + '/' + str(TopK)
                                    folder_path_metric_RS = os.path.join(folder_path_RS, metric)
                                    #print(folder_path_metric)
                                    folder_path_RS = os.path.join(folder_path_metric_RS, folder_name)        
                                    file_path_RS = os.path.join(folder_path_RS, filename)
                                    #print(file_path_RS)
                                    with open(file_path_RS, 'r') as csv_file:
                                        #print(file_path_RS)
                                        reader = csv.DictReader(csv_file)
                                        precision_sum = 0
                                        for row in reader:
                                            try:
                                                filename=str(row['IaC_file'])
                                                #print(filename)
                                                prec = float(row['PRECISION_average'])
                                                if prec >= 0.0:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    RS_precision.append(round(precision_sum,2))
                                                else:
                                                    precision_sum = prec
                                                    decimal_string=process_number(prec)
                                                    #decimal_string = str(precision_sum).split('.')[1][:2]
                                                    #print(decimal_string)

                                                    # Convert the decimal part to an integer
                                                    decimal_as_int = int(decimal_string)
                                                    RS_precision.append(round(precision_sum,2))    
                                            except (ValueError, KeyError):
                                                continue                                      



                                    
                                    
                                  
                                    seed=process_data(GA_precision, PSO_precision,RS_precision,SA_precision,HIST_precision,SIM_precision,commits,filename,co_modified_files,seed,csv_hist_path,csv_path_sim,csv_content_sim,test_commits,train_commits,project)
                                    GA_precision = []
                                    PSO_precision=[]
                                    RS_precision=[]
                                    commits = []
                                    co_modified_files = []
                                    success = []
                                        

iiab/roles/network/tasks/avahi.yml_(IaC)_
a450db1f1a5fafba73aa6b1c48e59ef4a3c91916


/var/folders/zg/r6wv5nm54bdd_pnz65ny98yh0000gn/T/ipykernel_6427/3675005689.py:2: DtypeWarning: Columns (158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,

4f94011781cd4b2a5d3ee6243ef37c3a981c96df
One of the files was not found: /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/iiab/roles/network/tasks/install.yml_(IaC)_ or /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/iiab/roles/network/tasks/avahi.yml


/var/folders/zg/r6wv5nm54bdd_pnz65ny98yh0000gn/T/ipykernel_6427/3675005689.py:2: DtypeWarning: Columns (158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,

iiab/roles/transmission/tasks/enable-or-disable.yml_(IaC)_
9f13454e8331843516d9f34be463a798d7f67611


...

/var/folders/zg/r6wv5nm54bdd_pnz65ny98yh0000gn/T/ipykernel_6427/3675005689.py:2: DtypeWarning: Columns (158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,

3f63c0b4dd407d75d283a868340828f34e2a1aed
One of the files was not found: /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/iiab/roles/mongodb/templates/iiab-mongodb-repair-if-no-lock.j2_(Other)_ or /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/iiab/roles/mongodb/tasks/install.yml
One of the files was not found: /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/iiab/roles/mongodb/templates/iiab-mongodb-repair-if-no-lock.j2_(Other)_ or /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/iiab/roles/mongodb/tasks/install.yml
One of the files was not found: /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/Experiment_6/Gits/iiab/roles/mongodb/templates/iiab-mongodb-repair-if-no-lock.j2_(Other)_ or /Users/narjes/Documents/ETS/Contrib5-Recommendation/APPROACHES/MainApproach_GA/

/var/folders/zg/r6wv5nm54bdd_pnz65ny98yh0000gn/T/ipykernel_6427/3675005689.py:2: DtypeWarning: Columns (158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,

iiab/roles/network/tasks/named.yml_(IaC)_
d12546c98df55fd870163ad9d020e7307d17d9ec


TypeError: Expected a valid Ansible script